In [1]:
## Tutorial for OpenSoundscape: training and testing CNNs

In [4]:
# setup

# Preprocessor classes are used to load, transform, and augment audio samples for use in a machine learing model
from opensoundscape.preprocess.preprocessors import BasePreprocessor, AudioToSpectrogramPreprocessor

# the cnn module provides classes for training/predicting with various types of CNNs
from opensoundscape.torch.models.cnn import Resnet18Multiclass, Resnet18Binary

#other utilities and packages
import torch
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
import random
import subprocess

#set up plotting
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize']=[15,5] #for large visuals
%config InlineBackend.figure_format = 'retina'

In [5]:
# set seeds for pytorch and python for reproducibility/debugging
torch.manual_seed(0)
random.seed(0)

In [6]:
# load data (once)
#subprocess.run(['curl','https://pitt.box.com/shared/static/79fi7d715dulcldsy6uogz02rsn5uesd.gz','-L', '-o','woodcock_labeled_data.tar.gz']) # Download the data
#subprocess.run(["tar","-xzf", "woodcock_labeled_data.tar.gz"]) # Unzip the downloaded tar.gz file
#subprocess.run(["rm", "woodcock_labeled_data.tar.gz"]) # Remove the file after its contents are unzipped


CompletedProcess(args=['rm', 'woodcock_labeled_data.tar.gz'], returncode=0)

In [7]:
# load table of labeled audiofiles 
#load Specky output: a table of labeled audio files
specky_table = pd.read_csv(Path("woodcock_labeled_data/woodcock_labels.csv"))
specky_table.head()

,filename,woodcock,sound_type
0,d4c40b6066b489518f8da83af1ee4984.wav,present,song
1,e84a4b60a4f2d049d73162ee99a7ead8.wav,absent,na
2,79678c979ebb880d5ed6d56f26ba69ff.wav,present,song
3,49890077267b569e142440fa39b3041c.wav,present,song
4,0c453a87185d8c7ce05c5c5ac5d525dc.wav,present,song


In [8]:
#update the paths to the audio files so that they are relative paths 
# - may want to use absolute paths instead of relative for your training? (my guess is no)
specky_table.filename = ['./woodcock_labeled_data/'+f for f in specky_table.filename]
specky_table.head()

,filename,woodcock,sound_type
0,./woodcock_labeled_data/d4c40b6066b489518f8da8...,present,song
1,./woodcock_labeled_data/e84a4b60a4f2d049d73162...,absent,na
2,./woodcock_labeled_data/79678c979ebb880d5ed6d5...,present,song
3,./woodcock_labeled_data/49890077267b569e142440...,present,song
4,./woodcock_labeled_data/0c453a87185d8c7ce05c5c...,present,song


In [ ]:
# use categorical_to_one_hot function from opensoundscape.annotations to create “one hot” labels
# (a column for every class, with 1 for present or 0 for absent) 
# In this case, our classes are simply 'negative' for files without a woodcock and 'positive' for files with a woodcock.

from opensoundscape.annotations import categorical_to_one_hot
#one_hot_labels, classes = categorical_to_one_hot(specky_table[['woodcock']].values)
#labels = pd.DataFrame(index=specky_table['filename'],data=one_hot_labels,columns=classes)
#labels.head()
